In [1]:
'''
# https://www.tensorflow.org/guide/keras/sequential_model
# Sequential 모델은 다음의 경우에 적합하지 않습니다.
# 모델에 다중 입력 또는 다중 출력이 있습니다
# 레이어에 다중 입력 또는 다중 출력이 있습니다
# 레이어 공유를 해야 합니다
# 비선형 토폴로지를 원합니다(예: 잔류 연결, 다중 분기 모델)

# Define Sequential model with 3 layers
model = keras.Sequential(
    [
        layers.Dense(2, activation="relu", name="layer1"),
        layers.Dense(3, activation="relu", name="layer2"),
        layers.Dense(4, name="layer3"),
    ]
)
# Call model on a test input
x = tf.ones((3, 3))
y = model(x)
'''
pass

In [2]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations

import os
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

* CNN
1. CNN은 전통적인 뉴럴 네트워크 앞에 여러 계층의 컨볼루셔널 계층을 붙인 모양
2. 컨볼루셔널 계층을 통해서 입력 받은 이미지에 대한 특징(Feature)를 추출하게 되고, 추출한 특징을 기반으로 기존의 뉴럴 네트워크를 이용하여 분류

* 컨볼루셔널 레이어(Convolutional Layer)
1. 입력 데이터로 부터 특징을 추출하는 역할
2. 특징을 추출하는 필터(Filter)
3. 필터의 값을 비선형 값으로 바꿔주는 액티베이션(Activiation) 함수

* 필터(Filter)
1. 특징이 데이터에 있는지 없는지 검출하는 함수
2. 필터는 구현에서 행렬로 정의
3. 입력 받는 이미지 모두 행렬로 변환
4. 입력받은 데이터에서 그 특징을 가지고 있으면 결과 값이 큰값, 특성을 가지고 있지 않으면 0에 가까운 값이 반환 -> 데이터가 그 특성을 가지고 있는지 없는지 여부

* strides
1. 필터를 적용하는 간격(예 우측으로 한칸씩 아래로 한칸씩 적용) 값
2. 필터를 적용해서 얻어낸 결과 Feature map 또는 activation map

* Padding
1. CNN 네트워크 특징 중 여러 단계에 걸쳐서 계속 필터를 연속적으로 적용하는데 필터 적용 후 결과 값이 작아지게 되면 처음에 비해 특징이 많이 유실 될 수 있음
2. 충분히 특징이 추출되기 전에 결과 값이 작아지면 특징이 유실되므로 이를 방지 하기 위해 사용하는 기법
3. 입력값 주위로 0값을 넣어서 입력 값의 크기를 인위적으로 키워 결과값이 작아지는 것을 방지

* Activation function
1. 필터를 통해서 추출한 값이 들어가 있는 예에서 큰값, 0에 가까운값들을 비선형 값으로 바꿔주는 과정이 필요함 -> Activation function
2. 예) 시그모이드(sigmoid)함수 : 뉴럴 네트워크에서 신경망이 깊어질수록 학습이 어렵기 때문에 전체 레이어를 한번 계산한 후 그 계산 값을 재활용하거나 다시 계산하는 Back propagation 방법을 사용하는데 sigmoid 함수는 propagation 제대로 작동하지 않음
3. sigmoid 대체 함수로 ReLu를 사용

* 풀링(Sub sampling or Pooling)
1. max pooling : Activation map을 MxN의 크기로 잘라낸 후, 그 안에서 가장 큰 값을 뽑아내는 방법
2. 값이 큰 값이 다른 특징들을 대표한다는 개념
3. 전체 데이터의 사이즈가 줄어들기 때문에 연산에 들어가는 리소스가 적어짐
4. 데이터의 크기를 줄이면서 소실이 발생하기 때문에 오버피팅을 방지

* BatchNormalization
1. 학습하는 과정 자체를 전체적으로 안정화시키고 학습 속도를 가속 시킬 수 있는 근본적인 방법을 제시하는 함수
2. 모델 업데이트


* Dropout
1. overfitting을 해소하기 위한 방법
2. 모델이 학습데이터에 대해 지나치게 훈련되어 실험데이터(test data)에 대해 결과가 좋지 못할 경우 동작




In [3]:
def get_sequential_model(input_shape):
    model = keras.Sequential(
        [
            # input
            layers.Input(input_shape),
            
            # 1st 
            layers.Conv2D(64, 3, strides=1, activation='relu', padding='same'),
            layers.Conv2D(64, 3, strides=1, activation='relu', padding='same'),
            layers.MaxPool2D(),
            layers.BatchNormalization(),
            layers.Dropout(0.5),
            
            # 2nd
            layers.Conv2D(128, 3, strides=1, activation='relu', padding='same'),
            layers.Conv2D(128, 3, strides=1, activation='relu', padding='same'),
            layers.MaxPool2D(),
            layers.BatchNormalization(),
            layers.Dropout(0.3),
            
            # Classifier
            layers.GlobalMaxPool2D(),
            layers.Dense(128, activation='relu'),
            layers.Dense(1, activation='sigmoid')
        ]
    )
    
    return model

input_shape = (256, 256, 3)
model = get_sequential_model(input_shape)

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 256, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 64)     0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 128, 128, 64)     256       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 128, 128, 64)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 128, 128, 128)     7

In [5]:
def get_functional_model(input_shape):
    inputs = keras.Input(input_shape)
    
    # 1st Conv block
    x = layers.Conv2D(64, 3, strides=1, activation='relu', padding='same')(inputs)
    x = layers.Conv2D(64, 3, strides=1, activation='relu', padding='same')(x)
    x = layers.MaxPool2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.5)(x)
    
    # 2nd Conv block
    x = layers.Conv2D(128, 3, strides=1, activation='relu', padding='same')(x)
    x = layers.Conv2D(128, 3, strides=1, activation='relu', padding='same')(x)
    x = layers.MaxPool2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dropout(0.3)(x)
    
    # Classifier
    x = layers.GlobalMaxPool2D()(x)
    x = layers.Dense(128, activation='relu')(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)
    
    model = keras.Model(inputs, outputs)
    
    return model
       

input_shape = (256, 256, 3)
model = get_sequential_model(input_shape)

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 256, 256, 64)      1792      
                                                                 
 conv2d_5 (Conv2D)           (None, 256, 256, 64)      36928     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 128, 128, 64)     0         
 2D)                                                             
                                                                 
 batch_normalization_2 (Batc  (None, 128, 128, 64)     256       
 hNormalization)                                                 
                                                                 
 dropout_2 (Dropout)         (None, 128, 128, 64)      0         
                                                                 
 conv2d_6 (Conv2D)           (None, 128, 128, 128)    

In [6]:
class SimpleCNN(keras.Model):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        
        self.conv_block_1 = keras.Sequential(
            [
                layers.Conv2D(64, 3, strides=1, activation='relu', padding='same'),
                layers.Conv2D(64, 3, strides=1, activation='relu', padding='same'),
                layers.MaxPool2D(),
                layers.BatchNormalization(),
                layers.Dropout(0.5)
            ], name='conv_block_1'
        )
        
        self.conv_block_2 = keras.Sequential(
            [
                layers.Conv2D(128, 3, strides=1, activation='relu', padding='same'),
                layers.Conv2D(128, 3, strides=1, activation='relu', padding='same'),
                layers.MaxPool2D(),
                layers.BatchNormalization(),
                layers.Dropout(0.3)
            ], name='conv_block_2'
        )
        
        self.classifier = keras.Sequential(
            [
                layers.GlobalMaxPool2D(),
                layers.Dense(128, activation='relu'),
                layers.Dense(1, activation='sigmoid')
            ], name='classifier'
        )
    
    def call(self, input_tensor, training=False):
        x = self.conv_block_1(input_tensor)
        x = self.conv_block_2(x)
        x = self.classifier(x)
        
        return x

In [7]:
input_shape = (None, 256, 256, 3)
model = SimpleCNN()
model.build(input_shape)

model.summary()

Model: "simple_cnn"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv_block_1 (Sequential)   (None, 128, 128, 64)      38976     
                                                                 
 conv_block_2 (Sequential)   (None, 64, 64, 128)       221952    
                                                                 
 classifier (Sequential)     (None, 1)                 16641     
                                                                 
Total params: 277,569
Trainable params: 277,185
Non-trainable params: 384
_________________________________________________________________


In [10]:
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics='accuracy'
)